In [14]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
%matplotlib inline 

In [4]:
input_shape = (224, 224, 3)
batch = 16
epochs = 30
lr = 0.002

In [5]:
img_gen = ImageDataGenerator(rotation_range=15,
                             zoom_range=0.15,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             rescale=1/255,
                             horizontal_flip=True,
                             fill_mode='nearest')

In [6]:
train_gen = img_gen.flow_from_directory('dataset/train',
                                       target_size=input_shape[:2],
                                       batch_size=batch,
                                       class_mode='categorical',
                                       shuffle=True,
                                       seed=42)
val_gen = img_gen.flow_from_directory('dataset/validation',
                                        target_size=input_shape[:2],
                                        batch_size=batch,
                                        class_mode='categorical',
                                        shuffle=True,
                                        seed=42)
test_gen = ImageDataGenerator().flow_from_directory('dataset/test',
                                                   target_size=input_shape[:2],
                                                   batch_size=batch,
                                                   class_mode='categorical',
                                                   shuffle=True,
                                                   seed=42)

Found 11540 images belonging to 3 classes.
Found 3553 images belonging to 3 classes.
Found 3574 images belonging to 3 classes.


In [7]:
train_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

In [8]:
test_gen.class_indices

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}

The main idea is to use pretrained model ResNet50 and then train it on the data that we provide.

In [15]:
# using pretrained model 
baseModel = InceptionV3(weights='imagenet', include_top=False)

# next step is to construct top of model to feed data
headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# actual model that we`ll be used
model = Model(inputs=baseModel.input, outputs=headModel)

# turn off the tuning of the base parametrs
for layer in baseModel.layers:
    layer.trainable = False
    
opt = Adam(lr=lr)
model.compile(loss="categorical_crossentropy", optimizer=opt,
             metrics=["accuracy"])

87916544/87910968 [==============================] - 10s 0us/step


In [16]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation_98 (Activation)      (None, None, None, 3 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
res = model.fit_generator(train_gen, 
                         steps_per_epoch=50,
                         epochs=epochs,
                         verbose=2,
                         validation_data=val_gen)

Epoch 1/30
